This notebook is a boilerplate notebooks with widgets for:
 - uploading a geojson/ shpaefile
 - downloading a processed artefact - raster
 - downloading a processed artefact - vector

In [78]:
from ipywidgets import widgets, IntSlider, jslink, HBox
from IPython.display import display, HTML
from ipyleaflet import Map, GeoData, ImageOverlay, basemaps, LayersControl, ScaleControl, FullScreenControl, WidgetControl
import geopandas as gpd
import io
import base64

import rioxarray
import xarray as xr
import xarray_leaflet
import matplotlib.pyplot as plt

In [79]:
test_raster = rioxarray.open_rasterio('evergreen-resources/data/temp/test-dem.tif')
test_raster = test_raster.rio.reproject('EPSG:4326')
test_raster = test_raster.sel(band=1)
#test_raster.plot()

In [80]:
#upload button
uploader = widgets.FileUpload(
    description='Upload file',
    accept='.geojson',
    multiple=False,
    button_style='primary'
)

#submit polygon button
submit = widgets.Button(
    description='Display AOI', 
    button_style='success', 
    tooltip='Click me')

#data processing/fetching button
get_data = widgets.Button(
    description='Get data for AOI', 
    button_style='primary', 
    tooltip='Click me')


 #note that the center coordinates are the opposite to what is stored in a geojson, you need to swap the lat/long
m = Map(height='500px', width='500px', 
          center=(-29.2253, 116.26012), zoom=10,
          basemap = basemaps.OpenTopoMap)



   fid                                           geometry
0    1  POLYGON ((116.26012 -29.22530, 116.26172 -29.2...


In [81]:

def get_vector(upload_widget):
    try:

        uploaded_filename = next(iter(upload_widget.value))
        content = upload_widget.value[uploaded_filename]['content']
        bytes_io = io.BytesIO(content)
        vector = gpd.read_file(bytes_io)
        print(vector)

    except Exception as e:
        print(e)
    return vector


def submit_clicked(b):
    try:
        fc = get_vector(uploader)
        geo_json = GeoData(geo_dataframe = fc, name='AOI')
        m.add_layer(geo_json)
    except Exception as e:
        print(e)
    

def print_file(b):
    fc = get_vector(uploader)
    print(fc.head())



In [82]:

buttons = HBox([uploader, submit, get_data])
display(buttons)

# zoom slider and widget control
zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=10)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widgetControl = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widgetControl)

m.add_layer(basemaps.OpenStreetMap.Mapnik)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl(position='topright'))
m.add_control(LayersControl())
display(m)

    
submit.on_click(submit_clicked)
get_data.on_click(print_file)

Map(center=[-29.2253, 116.26012], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [83]:
uploader.value

{}

In [84]:

# def create_download_link(df, title = "Download GeoDataFrame", filename = "data.geojson"): 
#     df.to_file(filename, driver='GeoJSON')
#     data = open(filename, "rb").read()
#     b64 = base64.b64encode(data)
#     payload = b64.decode()

#     html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
#     html = html.format(payload=payload,title=title,filename=filename)
#     return HTML(html)

# # create a link to download the dataframe
# download_button = create_download_link(test_raster)

In [85]:
#l = test_raster.leaflet.plot(Map, colormap=plt.cm.terrain)

In [86]:
#display(download_button)